In [1]:
from flask import Flask, Response, stream_with_context
import cv2
import requests
import numpy as np
from ultralytics import YOLO
import pandas as pd
from glob import glob
import matplotlib.pylab as plt
import torch

app = Flask(__name__)

In [2]:
model_type = "MiDaS_small"
midas = torch.hub.load('intel-isl/MiDaS', model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

# Input transformation pipeline
transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = transforms.dpt_transform
else:
    transform = transforms.small_transform

Using cache found in /Users/bigfatrat/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/bigfatrat/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /Users/bigfatrat/.cache/torch/hub/intel-isl_MiDaS_master


In [3]:
def find_largest_max_depth_area(depth_map):
    THRESHOLD = 100
    max_depth = np.max(depth_map)
    print("MAX DEPTH: ", max_depth)
    # Send the max depth value back to the Raspberry Pi server
    max_depth_op = float(max_depth)
    requests.post("http://192.168.107.54:5000/processing_done", json={"message": f"Max Depth is {max_depth_op}."})
    
    # binary mask (0,1)
    mask = depth_map >= max_depth - THRESHOLD
    mask = mask.astype(np.uint8)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # key=cv2.contourArea -> identifies the largest connected region 
    largest_contour = max(contours, key=cv2.contourArea) 
    
    # Get bounding box and centroid
    x, y, w, h = cv2.boundingRect(largest_contour)
    centroid_x = x + w // 2
    centroid_y = y + h // 2
    
    return max_depth, (centroid_x, centroid_y), contours



def visualize_largest_max_depth_area(depth_map, rgb_image):
    max_depth, centroid, contours = find_largest_max_depth_area(depth_map)
    
    for contour in contours:
        cv2.polylines(rgb_image, [contour], True, color=(0, 255, 0), thickness=2)
        # centroid
        cv2.circle(rgb_image, (centroid[0], centroid[1]), 5, (0, 255, 0), -1)
    
    return rgb_image

In [4]:
model = YOLO('../Crater-Detector/best.pt')
class_names = model.names

In [5]:
# URL to receive the video feed from Raspberry Pi
RPI_VIDEO_URL = "http://192.168.107.54:5000/video_feed"  # Replace with actual Raspberry Pi IP

In [6]:
def process_frame(frame):
    # Process frame for both predictions (depth and object detection)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)

    with torch.no_grad():
        depth_prediction = midas(imgbatch)
        depth_prediction = torch.nn.functional.interpolate(
            depth_prediction.unsqueeze(1), size=img.shape[:2], mode='bicubic', align_corners=False
        ).squeeze()
        depth_output = depth_prediction.cpu().numpy()


    # Normalize depth values to 0-255 for visualization
    output = (depth_output - depth_output.min()) / (depth_output.max() - depth_output.min()) * 255
    output = output.astype(np.uint8)
    output = cv2.applyColorMap(output, cv2.COLORMAP_WINTER)
    # cv2.imshow('Depth Map', output)
    
    # output_filename = os.path.join(output_dir, f'depth_frame.jpg')
    # cv2.imwrite(output_filename, output)
    
    image_with_depth = visualize_largest_max_depth_area(depth_output, img)
    
    results = model.predict(img.copy())  

    depth_at_center = 0
    angle = 0
    area = 0

    for r in results:
        boxes = r.boxes
        masks = r.masks

        if masks is not None:
            masks = masks.data.cpu()
            
            for seg, box in zip(masks.data.cpu().numpy(), boxes):
                seg = cv2.resize(seg, (img.shape[1], img.shape[0])) 
                contours, _ = cv2.findContours((seg).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
                # print(sorted_contours)
                for i, contour in enumerate(sorted_contours):
                    M = cv2.moments(contour)
                    if M["m00"] == 0:
                        continue
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    
                    d = int(box.cls)
                    c = class_names[d]
                    x, y, x1, y1 = cv2.boundingRect(contour)

                    depth_at_center = depth_output[cY, cX]  # Get depth at object center
                    cv2.putText(img, "Est.:{}".format(str(round(depth_at_center, 2))), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

                    cv2.polylines(img, [contour], True, color=(0, 0, 255), thickness=1)
                    cv2.putText(img, str(i) + c, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.circle(img, (cX, cY), 5, (0, 0, 255), -1)

                    ratio_pixel_mm = 155 / 14 # 14 mm = 155 pixels
                    mm = x1 / ratio_pixel_mm
                    cm = mm / 10
                    cv2.putText(img, str(round(cm, 2)), (x, y-30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 1)
                    
                    rect = cv2.minAreaRect(contour)
                    (mx,my),(mw,mh),angle = rect
                    cv2.putText(img, "Angle:{}".format(angle), (int(x), int(y)-40), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 255), 1 )
                    cv2.putText(img, "Area:{}".format(cv2.contourArea(contour)), (int(x), int(y)), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 255), 1 )
                    area = float(cv2.contourArea(contour))

    # Display results
    # cv2.imshow('Combined Prediction', img)
    processed_frame = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # output_filename = os.path.join(output_dir, f'output_frame.jpg')
    # cv2.imwrite(output_filename, img)            
    
    # Send the max depth value back to the Raspberry Pi server
    depth_at_center = float(depth_at_center)
    angle = float(angle)
    
    requests.post("http://192.168.107.54:5000/processing_done", json={"message": f"Est depth at center is {depth_at_center}, Angle is {angle}, Area is {area}"})
    
    return processed_frame

In [7]:
# import time 

def generate_processed_frames():
    stream = requests.get(RPI_VIDEO_URL, stream=True)
    byte_buffer = b''
    for chunk in stream.iter_content(chunk_size=1024):
        byte_buffer += chunk

        a = byte_buffer.find(b'\xff\xd8')  # Start of JPEG
        b = byte_buffer.find(b'\xff\xd9')  # End of JPEG

        if a != -1 and b != -1:
            jpg = byte_buffer[a:b+2]
            byte_buffer = byte_buffer[b+2:]

            # Decode frame and process
            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            processed_frame = process_frame(frame)

            # Send confirmation message back to Raspberry Pi after processing
            requests.post("http://192.168.107.54:5000/processing_done", json={"message": "Processing done for this frame"})

            # Encode the processed frame back to JPEG
            ret, buffer = cv2.imencode('.jpg', processed_frame)
            if not ret:
                continue

            processed_frame = buffer.tobytes()

            # time.sleep(2)

            # Yield the processed frame as bytes
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + processed_frame + b'\r\n')

In [8]:
@app.route('/')
def index():
    # Automatically redirect to /processed_feed
    return Response("Redirecting to /processed_feed", status=302, headers={'Location': '/processed_feed'})

@app.route('/processed_feed')
def processed_feed():
    return Response(stream_with_context(generate_processed_frames()), mimetype='multipart/x-mixed-replace; boundary=frame')

In [ ]:
app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.107.172:5001
Press CTRL+C to quit
192.168.107.172 - - [25/Oct/2024 15:36:49] "GET / HTTP/1.1" 302 -


MAX DEPTH:  839.0211

0: 480x640 5 Boulders, 4 craters, 179.8ms
Speed: 1.5ms preprocess, 179.8ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)


192.168.107.172 - - [25/Oct/2024 15:36:50] "GET /processed_feed HTTP/1.1" 200 -


MAX DEPTH:  803.971

0: 480x640 1 Boulder, 5 craters, 239.7ms
Speed: 1.0ms preprocess, 239.7ms inference, 16.8ms postprocess per image at shape (1, 3, 480, 640)
MAX DEPTH:  844.79755

0: 480x640 4 Boulders, 5 craters, 144.2ms
Speed: 1.1ms preprocess, 144.2ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)
MAX DEPTH:  868.0027

0: 480x640 2 Boulders, 6 craters, 151.9ms
Speed: 1.4ms preprocess, 151.9ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
MAX DEPTH:  869.86755

0: 480x640 2 Boulders, 6 craters, 144.3ms
Speed: 1.1ms preprocess, 144.3ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)
MAX DEPTH:  837.4005

0: 480x640 1 Boulder, 3 craters, 145.0ms
Speed: 1.0ms preprocess, 145.0ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
MAX DEPTH:  894.78894

0: 480x640 5 Boulders, 4 craters, 142.4ms
Speed: 1.3ms preprocess, 142.4ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)
MAX DEPTH:  876.6879

0: 48